# Asignment 1b :Task 1

Assignment 1 Question 2 of stanford course, Tensorflow for Deep Learning Research. 
Assignment available here https://web.stanford.edu/class/cs20si/2017/assignments/a1.pdf

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import xlrd
import math
import time

## Question 2, Task 1: Improving the accuracy of our logistic regression on MNIST
We got the accuracy of ~90% on our MNIST dataset with our vanilla model, which is
unacceptable. The dataset is basically solved and state of the art models reach accuracies above
99%. You can use whatever loss functions, optimizers, even models that you want, as long as
your model is built in TensorFlow. You can save your code in the file q2.py. In the comments,
explain what you decided to do, instruction on how to run your code, and report your results.
I’m happy if you can reach 97%.

In [2]:
# Step 1: Read in Data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/data/mnist',one_hot=True)

Extracting /data/mnist\train-images-idx3-ubyte.gz
Extracting /data/mnist\train-labels-idx1-ubyte.gz
Extracting /data/mnist\t10k-images-idx3-ubyte.gz
Extracting /data/mnist\t10k-labels-idx1-ubyte.gz


## Linear Regression with different regulrization

In [51]:
# Defing Model
# Define parameters 
tf.reset_default_graph()
learning_rate = 0.1
batch_size = 128
n_epochs = 200
lamda = 0.0001
graph1 = tf.Graph()
with graph1.as_default():
    #Step 2: 
    X = tf.placeholder(tf.float32,[batch_size,784],name='X_placeholder')
    Y = tf.placeholder(tf.float32,[batch_size,10],name='Y_placeholder')
    #Step 3:
    weights = tf.Variable(tf.random_normal(shape=[784,10],stddev=0.01),name='weights') 
    bias = tf.Variable(tf.zeros([1,10]),name='bias')

    #Step 4:
    logits = tf.matmul(X,weights) + bias

    #Step 5: 
    entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=logits,name='loss')
    loss = tf.reduce_mean(entropy) #+ lamda*tf.norm(weights,ord=np.inf)
    # Step 6:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [52]:
# Run Session
tf.reset_default_graph()
with tf.Session(graph=graph1) as sess:
    writer = tf.summary.FileWriter('./graphs/assignment1_b',sess.graph)
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    n_batches = int(mnist.train.num_examples/batch_size)
    for i in range(n_epochs):
        total_loss = 0
        for j in range(n_batches):
            X_batch,Y_batch = mnist.train.next_batch(batch_size)
            _,loss_batch = sess.run([optimizer,loss],feed_dict={X:X_batch,Y:Y_batch})
            total_loss +=loss_batch
        print('Average loss epoch {0}:{1}'.format(i,total_loss/n_batches))    
    print('Total_time:{0} seconds',format(time.time()-start_time))
# Test The model
    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        loss_batch,logits_batch = sess.run([loss,logits],feed_dict={X:X_batch,Y:Y_batch})
        probs = tf.nn.softmax(logits_batch)
        correct_preds = tf.equal(tf.argmax(probs,1),tf.argmax(Y_batch,1))
        n_correct_preds = tf.reduce_sum(tf.cast(correct_preds,tf.float32))
        total_correct_preds += sess.run(n_correct_preds)
    print('Accuracy{0}'.format(total_correct_preds/mnist.test.num_examples))    
writer.close()           

Average loss epoch 0:0.5882752966352832
Average loss epoch 1:0.38106328134353346
Average loss epoch 2:0.3516148044999107
Average loss epoch 3:0.3316947699704648
Average loss epoch 4:0.32142395020781694
Average loss epoch 5:0.3102460809243031
Average loss epoch 6:0.30729146826239456
Average loss epoch 7:0.29900672604570855
Average loss epoch 8:0.29976612405899244
Average loss epoch 9:0.2923300792148341
Average loss epoch 10:0.2918933412898115
Average loss epoch 11:0.28744025709051074
Average loss epoch 12:0.2852834333559294
Average loss epoch 13:0.28602139738611965
Average loss epoch 14:0.2825078365060833
Average loss epoch 15:0.2790428126330698
Average loss epoch 16:0.28291179194594873
Average loss epoch 17:0.27751066933422935
Average loss epoch 18:0.2754802831333556
Average loss epoch 19:0.2756789408467866
Average loss epoch 20:0.2738904013083531
Average loss epoch 21:0.27017483010992305
Average loss epoch 22:0.2769439657737603
Average loss epoch 23:0.2691879564231926
Average loss epo

## Results

- 0.924 No Regulrization 
- l_1-regulrization 
 - 0.46 with lambda=0.1
 - 0.8 with lambda=0.01
 - 0.9015 with lambdba = 0.001 


- l_2-regulrization 
 - 0.911 with lambda=0.01
 
 
- l_$\inf$ regulrization
 - 0.9191 with lambda=0.1
 - 0.9225 with lambda=0.01
 - 0.9212 with lambda=0.0001
 - Note: Trainig Loss decreses rapidly and goes down to 0.26 test accuracy = 0.9212 

## Neural Network

Also experimented on DRAC1. Seems worse. A paper "[Generlization in Deep Learning](https://arxiv.org/abs/1710.05468)" introduces a new regulrization method they call DRAC1. For more details, read section 5 of the paper. 

In [53]:
# To avoid haveing different graphs colliding
tf.reset_default_graph()
learning_rate = 0.1
batch_size = 128
n_epochs = 200
lamda = 0.0001
weight_initializer_xavier = tf.contrib.layers.xavier_initializer()
# Neurons in each layer
X_shape_0 = 784 # X.shape[0]
n_outputs = 10
Neurons_1 = 100
Neurons_2 = 50

# graph 
graph1 = tf.Graph()
with graph1.as_default():
    #plcae holders for input: 
    X = tf.placeholder(tf.float32,[batch_size,X_shape_0],name='X_placeholder')
    Y = tf.placeholder(tf.float32,[batch_size,n_outputs],name='Y_placeholder')
    # Layer 1
    w1 = tf.Variable(weight_initializer_xavier(shape=[X_shape_0,Neurons_1]),name='w1') 
    b1 = tf.Variable(tf.zeros([1,Neurons_1]),name='b1')
    z1 = tf.matmul(X,w1) + b1
    a1 = tf.nn.relu(z1)
    # Layer 2
    w2 = tf.Variable(weight_initializer_xavier(shape=[Neurons_1,Neurons_2]),name='w2') 
    b2 = tf.Variable(tf.zeros([1,Neurons_2]),name='b2')
    z2 = tf.matmul(a1,w2) + b2
    a2 = tf.nn.elu(z2)
    #Output Layer
    w3 = tf.Variable(weight_initializer_xavier(shape=[Neurons_2,n_outputs]),name='w3') 
    b3 = tf.Variable(tf.zeros([1,n_outputs]),name='b3')
    z3 = tf.matmul(a2,w3) + b3
    a3 = tf.nn.softmax(z3)
    
    entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=a3,name='loss')
    loss = tf.reduce_mean(entropy) 
    #loss_DRAC1 = loss+ (lamda/batch_size)*tf.reduce_sum(tf.reduce_max(tf.abs(a3),axis=1))
    #loss_l1 = loss + 0.001* tf.norm(w3,ord=1)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [54]:
# Run Session
tf.reset_default_graph()
with tf.Session(graph=graph1) as sess:
    # to save the graph
    #writer = tf.summary.FileWriter('./graphs/assignment1_b',sess.graph)
    #start_time = time.time()
    sess.run(tf.global_variables_initializer())
    n_batches = int(mnist.train.num_examples/batch_size)
    
    #Training 
    for i in range(n_epochs):
        total_loss = 0
        # Passing through each batch
        for j in range(n_batches):
            X_batch,Y_batch = mnist.train.next_batch(batch_size)
            _,loss_batch = sess.run([optimizer,loss],feed_dict={X:X_batch,Y:Y_batch})
            total_loss +=loss_batch
        print('Average loss epoch {0}:{1}'.format(i,total_loss/n_batches))    
    print('Total_time:{0} seconds',format(time.time()-start_time))
    
    # Testing
    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        loss_batch,logits_batch = sess.run([loss,a3],feed_dict={X:X_batch,Y:Y_batch})
        probs = tf.nn.softmax(logits_batch)
        correct_preds = tf.equal(tf.argmax(probs,1),tf.argmax(Y_batch,1))
        n_correct_preds = tf.reduce_sum(tf.cast(correct_preds,tf.float32))
        total_correct_preds += sess.run(n_correct_preds)
    print('Accuracy{0}'.format(total_correct_preds/mnist.test.num_examples))    
#writer.close()    

Average loss epoch 0:1.9099238896981263
Average loss epoch 1:1.6786988423420832
Average loss epoch 2:1.647966335703443
Average loss epoch 3:1.6365570746935332
Average loss epoch 4:1.6286025164010642
Average loss epoch 5:1.624580172114161
Average loss epoch 6:1.6202697284293897
Average loss epoch 7:1.6162624578653793
Average loss epoch 8:1.6119685064662586
Average loss epoch 9:1.6098286444212728
Average loss epoch 10:1.6074868037706211
Average loss epoch 11:1.6052691389074971
Average loss epoch 12:1.6019936988959502
Average loss epoch 13:1.6023362499850613
Average loss epoch 14:1.5777154079961888
Average loss epoch 15:1.5226610223730128
Average loss epoch 16:1.517131548661452
Average loss epoch 17:1.513735964581683
Average loss epoch 18:1.5114402448658621
Average loss epoch 19:1.5085594784685503
Average loss epoch 20:1.5067970299776339
Average loss epoch 21:1.5068086969268906
Average loss epoch 22:1.5030077417969425
Average loss epoch 23:1.5025144151318601
Average loss epoch 24:1.500814

### Results 

3 Layer Neural Network, with N1=50, N2=10 accuracy was less than linear regression
- Layer 1 Neurons=100 Layer 2 Neurons = 40 or 50, epochs=30
 - no reg   accuracy = 0.8664 
 - l1-norm  accuracy = 95
 - <span style= "color:red"> DRAC1    accuracy = 0.8559 </span>
 - Note: Training loss was not decreasing very fast
- Layer 1 Neurons=100 Layer 2 Neurons = 50, epochs=200
 - no reg   accuracy = 0.9739
 - l1-norm  accuracy = 0.9716
 - <span style="color:red"> DRAC1    accuracy = 0.8873 (??) 1000 seconds extra time </span>
 